<a href="https://colab.research.google.com/github/Ashad001/DiseaseDiagnosis/blob/main/Disease_Diagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cython
!pip install pomegranate==v0.14.9


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 15.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Created wheel for pomegranate: filename=pomegranate-0.14.9-cp310-cp310-linux_x86_64.whl size=18331557 sha256=83de5e120860f21e8501025073fd98acb4cddce9f648bbf5531206ad7304f0b0
  Stored in directory: /root/.cache/pip/wheels/14/e7/b2/189a2d351ac4ae073cfa17ce9d56936d59af5712a18028fc31
Successfully built pomegranate
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.10
    Uninstalling Cython-3.0.10:
      Successfully uninstalled Cython-3.0.10


In [2]:
from pomegranate import *

desease = DiscreteDistribution({"RareX": 0.1, "CommonY": 0.5, "UnCommonY": 0.4}) # 0.1 + 0.5 + 0.4 = 1
age_group = DiscreteDistribution({"Young": 0.2, "Middle": 0.5, "Old": 0.3}) # 0.2 + 0.5 + 0.3 = 1
gender_group = DiscreteDistribution({"Male": 0.5, "Female": 0.5}) # 0.5 + 0.5 = 1


# P(Fever | Disease, agr)
fever = ConditionalProbabilityTable([
    ["RareX", "Young", "True", 0.8],
    ["RareX", "Young", "False", 0.2], # 0.8 + 0.2 = 1
    ["RareX", "Middle", "True", 0.7],
    ["RareX", "Middle", "False", 0.3], # 0.7 + 0.3 = 1 and so on
    ["RareX", "Old", "True", 0.6],
    ["RareX", "Old", "False", 0.4],
    ["CommonY", "Young", "True", 0.4],
    ["CommonY", "Young", "False", 0.6],
    ["CommonY", "Middle", "True", 0.3],
    ["CommonY", "Middle", "False", 0.7],
    ["CommonY", "Old", "True", 0.2],
    ["CommonY", "Old", "False", 0.8],
    ["UnCommonY", "Young", "True", 0.5],
    ["UnCommonY", "Young", "False", 0.5],
    ["UnCommonY", "Middle", "True", 0.4],
    ["UnCommonY", "Middle", "False", 0.6],
    ["UnCommonY", "Old", "True", 0.3],
    ["UnCommonY", "Old", "False", 0.7]
], [desease, age_group])

# P(Rash | Disease, Gender)
rash = ConditionalProbabilityTable([
    ["UnCommonY", "Male", "True", 0.8],  # 0.8 + 0.2 = 1
    ["UnCommonY", "Female", "False", 0.2],
    ["UnCommonY", "Male", "False", 0.7],
    ["UnCommonY", "Female", "True", 0.3],
    ["CommonY", "Male", "False", 0.6],
    ["CommonY", "Male", "True", 0.4],
    ["CommonY", "Female", "True", 0.5],
    ["CommonY", "Female", "False", 0.5],
    ["RareX", "Male", "False", 0.4],
    ["RareX", "Male", "True", 0.6],
    ["RareX", "Female", "True", 0.3],
    ["RareX", "Female", "False", 0.7],
], [desease, gender_group])

# P(Joint Pain | Disease)
joint_pain = ConditionalProbabilityTable([
    ["RareX", "True", 0.8],
    ["RareX", "False", 0.2],
    ["CommonY", "True", 0.6],
    ["CommonY", "False", 0.4],
    ["UnCommonY", "True", 0.4],
    ["UnCommonY", "False", 0.6]
], [desease])


# Make states
s1 = State(desease, name="Disease")
s2 = State(age_group, name="Age")
s3 = State(gender_group, name="Gender")
s4 = State(fever, name="Fever")
s5 = State(rash, name="Rash")
s6 = State(joint_pain, name="Joint Pain")

fever_network = BayesianNetwork("Fever Diagnosis")
fever_network.add_states(s1, s2, s4) # S4 is the fever and below is following the expression # P(Fever | Disease, age)
fever_network.add_edge(s1, s4)
fever_network.add_edge(s2, s4)
# fever_network.add_edge(s3, s4)
fever_network.bake()

import numpy as np
# Below line shows a young male having a fever with RareX disease
X = ["RareX", "Young", "True"]
X = np.array(X, ndmin=2) # Changing the X to a 2 dim vector (https://github.com/jmschrei/pomegranate/issues/672)
pred = fever_network.probability(X)
print(f"Probability of a young male having fever: {pred}")

rash_network = BayesianNetwork("Rash Diagnosis")
rash_network.add_states(s1, s3, s5) # S5 is Rash and below is following the expression # P(Rash | Disease, gender)
rash_network.add_edge(s1, s5)
rash_network.add_edge(s3, s5)
rash_network.bake()

import numpy as np
# Below line shows a female not having a rash with CommonY disease
X = ["CommonY", "Female", "False"]
X = np.array(X, ndmin=2)
pred = rash_network.probability(X)
print(f"Probability of not having a rash given a female having CommonY Disease: {pred}")




Probability of a young male having fever: 0.01600000000000001
Probability of not having a rash given a female having CommonY Disease: 0.12500000000000003
